In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans, AffinityPropagation, MeanShift, SpectralClustering, OPTICS, Birch
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, RobustScaler, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, rand_score, adjusted_rand_score, completeness_score, fowlkes_mallows_score, homogeneity_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/Users/nickdimmitt/Desktop/finance/data/stocks_clean.csv")

In [ ]:
df = df.loc[:,~df.columns.str.contains("Unnamed")]

## Feature Addition

In [ ]:
df['year_2'] = df.groupby('ticker').cumcount()+1

In [ ]:
df = df.replace(-9999, np.nan)
df = df.replace(-np.Inf, np.nan)
df = df.replace(np.Inf, np.nan)

In [ ]:
## market cap
mkt_cap = pd.read_csv("/Users/nickdimmitt/Desktop/finance/data/market_cap.csv")
mkt_cap['ticker'] = mkt_cap['Symbol']
mkt_cap = mkt_cap.drop('Symbol', axis=1)

df = df.merge(right=mkt_cap, how='inner', on='ticker')

columns = {
    'Market Cap':'mkt_cap',
    'Country':'country',
    'IPO Year':'ipo_year',
    'Secter': 'sector'}
df = df.rename(columns=columns)

df.head()

In [ ]:
df = df[~df['mkt_cap'].isna()]
df["Sector"] = df["Sector"].fillna("N/A")
df = df.fillna(-1)
df[df['year'] == 2023]['mkt_cap'] = 0

In [ ]:
df.to_csv("/Users/nickdimmitt/Desktop/finance/data/cluster_df.csv")

## Clustering

In [99]:
mkt_cap = pd.read_csv("/Users/nickdimmitt/Desktop/finance/data/market_cap.csv")
df = pd.read_csv("/Users/nickdimmitt/Desktop/finance/data/cluster_df.csv")

In [108]:
columns = {
    'Symbol': 'ticker',
    'Market Cap':'mkt_cap',
    'Country':'country',
    'IPO Year':'ipo_year',
    'Secter': 'sector'}
mkt_cap = mkt_cap.rename(columns=columns)

df = pd.merge(df, mkt_cap[['ticker', 'mkt_cap']], on='ticker')

In [115]:
df.head()

,Unnamed: 0,year,revenue,cogs,gross_profit,gross_profit_ratio,operating_expenses,r_&_d_expenses,selling_g_&_a_exp,general_and_admin_exp,...,cash_flow_cov,roa,roe,return_on_invested_capital,gross_profit_margin,ebitda_margin,net_profit_ratio,asset_turnovers,sector_trans,mkt_cap_y
0,0,1998.0,7952.0,4035.0,3917.0,0.4926,3475.0,948.0,2050.0,2050.0,...,0.213258,0.785442,1.296161,0.088630,15.241245,3.396887,15.241245,0.051534,6,4.015413e+10
1,1,1999.0,8331.0,4388.0,3943.0,0.4733,3202.0,997.0,2205.0,2205.0,...,0.016062,0.724284,1.165878,0.136113,7.701172,2.464844,7.701172,0.094048,6,4.015413e+10
2,2,2000.0,10773.0,5522.0,5251.0,0.4874,4198.0,1258.0,2940.0,2940.0,...,0.005076,0.623264,0.997341,0.124985,6.936592,2.191546,6.936592,0.089852,6,4.015413e+10
3,3,2001.0,8396.0,5166.0,3230.0,0.3847,4008.0,1349.0,2659.0,2659.0,...,0.310190,0.404458,0.570772,-0.097420,19.226190,4.946429,19.226190,0.021037,6,4.015413e+10
4,4,2002.0,6010.0,3694.0,2316.0,0.3854,3923.0,1169.0,2754.0,2754.0,...,-0.366346,0.282336,0.500540,-0.195904,-2.244186,0.796512,-2.244186,-0.125808,6,4.015413e+10


### Add in Ratios

In [109]:
df['div_payout_ratio'] = df['dividends_paid']/df['net_income']

df['inventory_turnover_ratio'] = df['cogs']/df['inventory']

df['current_ratio'] = df['total_current_assets']/df['total_current_liabilities']

df['quick_ratio'] = (df['total_current_assets'] - df['inventory'])/df['total_current_liabilities']

df['debt_ratio'] = df['total_liabilities']/df['total_assets']

df['debt_to_worth'] = df['total_liabilities']/df['total_stockholders_equity']

df['equity_ratio'] = df['total_stockholders_equity']/df['total_assets']

df['debt_to_tangible_net_worth'] = df['total_liabilities']/df['net_income']

df['interest_cov'] = df['ebitda']/df['interest_expense']

df['cash_flow_cov'] = df['free_cash_flow']/df['total_current_liabilities']

df['roa'] = df['gross_profit']/df['total_assets']

df['roe'] = df['gross_profit']/df['total_stockholders_equity']

df['return_on_invested_capital'] = df['operating_income']/(df['total_liabilities'] + df['total_stockholders_equity'])

df['gross_profit_margin'] = df['gross_profit']/df['net_income']

df['ebitda_margin'] = df['ebitda']/df['net_income']

df['net_profit_ratio'] = df['gross_profit']/df['net_income']

df['asset_turnovers'] = df['net_income']/df['total_assets']

In [126]:
df = df.replace(np.inf, 1)
df = df.replace(-np.inf, -1)
df = df.fillna(-1)
df['Sector'] = df['Sector'].replace(-1, "N/A")

In [127]:
features = ['mkt_cap_y','gross_profit_ratio','operating_income_ratio',
       'income_before_tax_ratio',
       'net_income_ratio', 'eps', 'eps_diluted', 'div_payout_ratio', 'inventory_turnover_ratio',
       'current_ratio', 'quick_ratio', 'debt_ratio', 'debt_to_worth',
       'equity_ratio', 'debt_to_tangible_net_worth', 'interest_cov',
       'cash_flow_cov', 'roa', 'roe', 'return_on_invested_capital',
       'gross_profit_margin', 'ebitda_margin', 'net_profit_ratio',
       'asset_turnovers', 'year_2', 'sector_trans']

In [128]:
le = LabelEncoder()
label = le.fit_transform(df['Sector'])
df['sector_trans'] = label

X = df[features]

X = RobustScaler().fit_transform(X)

In [129]:
kmeans = KMeans(n_clusters=2000)
df['labels'] = kmeans.fit_predict(X)

In [136]:
df[df['ticker'] == 'MSFT']

,Unnamed: 0,year,revenue,cogs,gross_profit,gross_profit_ratio,operating_expenses,r_&_d_expenses,selling_g_&_a_exp,general_and_admin_exp,...,roa,roe,return_on_invested_capital,gross_profit_margin,ebitda_margin,net_profit_ratio,asset_turnovers,sector_trans,mkt_cap_y,labels
60406,60482,1986.0,198.0,35.0,162.0,0.8223,102.0,-1.0,96.0,-1.0,...,0.947368,1.165468,0.358824,4.153846,1.846154,4.153846,0.228070,10,1.850455e+12,1070
60407,60483,1987.0,346.0,66.0,280.0,0.8083,153.0,-1.0,145.0,-1.0,...,0.972222,1.171548,0.440972,3.888889,1.791667,3.888889,0.250000,10,1.850455e+12,1070
60408,60484,1988.0,591.0,132.0,459.0,0.7766,271.0,-1.0,255.0,-1.0,...,0.931034,1.220745,0.378543,3.701613,1.612903,3.701613,0.251521,10,1.850455e+12,1070
60409,60485,1989.0,804.0,180.0,624.0,0.7760,381.0,-1.0,357.0,-1.0,...,0.865465,1.110320,0.335645,3.649123,1.608187,3.649123,0.237171,10,1.850455e+12,1070
60410,60486,1990.0,1183.0,206.0,977.0,0.8256,584.0,-1.0,538.0,-1.0,...,0.884163,1.063112,0.355335,3.501792,1.637993,3.501792,0.252489,10,1.850455e+12,1070
60411,60487,1991.0,1843.0,287.0,1557.0,0.8444,907.0,-1.0,831.0,-1.0,...,0.947080,1.152480,0.395377,3.362851,1.611231,3.362851,0.281630,10,1.850455e+12,1070
60412,60488,1992.0,2759.0,354.0,2405.0,0.8716,1409.0,-1.0,1296.0,-1.0,...,0.910985,1.096671,0.377273,3.396893,1.629944,3.396893,0.268182,10,1.850455e+12,1070
60413,60489,1993.0,3753.0,482.0,3271.0,0.8716,1945.0,-1.0,1794.0,-1.0,...,0.859658,1.008945,0.348489,3.432319,1.628541,3.432319,0.250460,10,1.850455e+12,1070
60414,60490,1994.0,4649.0,526.0,4123.0,0.8869,2397.0,610.0,1550.0,-1.0,...,0.768786,0.926517,0.321835,3.597731,1.709424,3.597731,0.213686,10,1.850455e+12,1070
60415,60491,1995.0,5937.0,608.0,5329.0,0.8976,3291.0,860.0,2162.0,-1.0,...,0.739112,0.999250,0.287650,3.667584,1.676531,3.667584,0.201526,10,1.850455e+12,1070


In [139]:
df[df['labels'] == 1927][['ticker', 'year_2', 'Sector']]

,ticker,year_2,Sector
302,AAPL,24,Technology
303,AAPL,25,Technology
3215,AIG,15,Finance
5070,AMOV,9,Telecommunications
6098,APA,23,Energy
...,...,...,...
88709,TEF,10,Telecommunications
88711,TEF,12,Telecommunications
91756,TSM,10,Technology
92587,TXN,22,Technology


### K-Means

In [ ]:
Sum_of_squared_distances = []
K = range(1,2000,75)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(X)
    print(km.inertia_)
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [10]:
final = pd.DataFrame()
for x in range(250,500,10):
    kmeans = KMeans(n_clusters=x)
    df['labels'] = kmeans.fit_predict(X)


    labels = list(df[df['year'] == 2022]['labels'].unique())
    df['proj_mktcap_lbl'] = df.groupby('labels')['mkt_cap'].transform(lambda x:x.mean())
    final = pd.concat([final, df[df['labels'].isin(labels)]])

### Ward Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import silhouette_score

range_n_clusters = range(100,1000,100)
silhouette_avg = []
for num_clusters in range_n_clusters:
 
 # initialise kmeans
    ward = AgglomerativeClustering(n_clusters=num_clusters, linkage='single')
    ward.fit(X)
    cluster_labels = ward.labels_
    print(num_clusters)
 # silhouette score
    silhouette_avg.append(silhouette_score(X, cluster_labels))
    
plt.plot(range_n_clusters,silhouette_avg,'bx-')
plt.xlabel('Values of K') 
plt.ylabel('Silhouette score') 
plt.title('Silhouette analysis For Optimal k')
plt.show()

In [11]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import silhouette_score


clusters = range(50,200,25)
for x in clusters:
    ward = AgglomerativeClustering(n_clusters=x, linkage='single')
    df['labels'] = ward.fit_predict(X)

    labels = list(df[df['year'] == 2022]['labels'].unique())
    df['proj_mktcap_lbl'] = df.groupby('labels')['mkt_cap'].transform(lambda x:x.mean())
    final = pd.concat([final, df[df['labels'].isin(labels)]])

In [12]:
final = final[final['year'] == 2022]
final['proj_mktcap_comp'] = final.groupby(['ticker'])['proj_mktcap_lbl'].transform(lambda x:x.mean())
final = final.drop_duplicates(subset='ticker')
##final.sort_values(by='proj_mktcap_comp', ascending=False).to_csv("/Users/nickdimmitt/Desktop/finance/data/clustered.csv")

In [85]:
columns = {
    'Symbol': 'ticker',
    'Market Cap':'mkt_cap',
    'Country':'country',
    'IPO Year':'ipo_year',
    'Secter': 'sector'}
mkt_cap = mkt_cap.rename(columns=columns)

In [14]:
final = pd.merge(final, mkt_cap, on='ticker')

In [23]:
final.head()

,Unnamed: 0,year,revenue,cogs,gross_profit,gross_profit_ratio,operating_expenses,r_&_d_expenses,selling_g_&_a_exp,general_and_admin_exp,...,net_profit_ratio,asset_turnovers,sector_trans,labels,proj_mktcap_lbl,proj_mktcap_comp,mkt_cap_y,country_y,ipo_year_y,Sector_y
0,24,2022.0,6848.0,3126.0,3722.0,0.5435,2104.0,467.0,1637.0,-1.0,...,2.968102,0.119066,6,53,1.315146e+10,1.295524e+10,4.015413e+10,United States,1999.0,Industrials
1,34,2022.0,12451.0,10212.0,2239.0,0.1798,1516.0,32.0,204.0,-1.0,...,-21.950980,-0.006900,6,53,1.315146e+10,1.295524e+10,7.947682e+09,NaN,2016.0,Industrials
2,58,2022.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0,...,0.250000,-0.026490,6,53,1.315146e+10,1.295524e+10,2.079234e+08,United States,2021.0,Industrials
3,133,2022.0,48971.0,15186.0,33785.0,0.6899,32178.0,-1.0,1815.0,-1.0,...,266.023622,0.001962,1,235,6.742056e+10,6.775593e+10,1.005893e+10,United States,NaN,Consumer Discretionary
4,254,2022.0,889.0,651.0,238.0,0.2673,111.0,-1.0,111.0,-1.0,...,2.380000,0.122850,6,53,1.315146e+10,1.295524e+10,4.813494e+09,United States,NaN,Industrials


In [65]:
final['proj_mktcap_comp'] = round(final['proj_mktcap_comp']/1000,2)
final['mktcap'] = round(final['mkt_cap_y']/1000,2)

In [25]:
final['difference'] = final['proj_mktcap_comp'] - final['mktcap']

In [29]:
final[['ticker', 'ipo_year', 'Sector', 'proj_mktcap_comp']].sort_values(by='proj_mktcap_comp', ascending=False).to_csv("clustered_1.csv")

In [ ]:
final.sort_values("difference", ascending=False).head(15)

In [70]:
final = final[final['proj_mktcap_comp'] != 12.96]

In [76]:
final.sort_values('difference', ascending=False)[['ticker', 'Sector_y', 'ipo_year_y', 'mktcap', 'country_y']].to_csv("cluster_3.csv")

12.96     2103
12.83       22
14.01       18
30.22       18
15.13       16
          ... 
128.77       1
35.49        1
7.15         1
3.10         1
10.10        1
Name: proj_mktcap_comp, Length: 214, dtype: int64